<a href="https://colab.research.google.com/github/dauten/DeepLearning/blob/develop/Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import 

In [0]:
def myTestFunc(test):
  return("The text is: " + str(test))

In [0]:
myTestFunc("oranges")
myTestFunc()

'The text is: 5'